# Lecture 15 - Mechanistic modeling

In this lecture you learned about mechanistic modeling and, in particular, one type of mathematical modeling approach based on ordinary differential equations (ODEs). 

In this tutorial you will learn how to create and simulate ODE-based models in python. 

> It is an ambitious tutorial. 😇 It is better to understand each exercise carefully than rushing to get to the end.

### Learning objectives:

- Write ODEs using python functions
- Integrate ODEs using *scipy*
- Plot simulation results
- Calibrate a model using experimental data

-------

## Exercise 0: warm-up

Let's assume we have a system with two simple chemical reactions: 
 - $R_1: A \to B$ with the kinetic rate: $r_1 = k_1 A$
 - $R_2: B \to C$ with the kinetic rate: $r_2 = k_2 B$
 
This leads to the following ODE model:

 - $\frac{dA}{dt} = -r_1$ 
 - $\frac{dB}{dt} = r_1 - r_2$
 - $\frac{dC}{dt} = r_2$ 
 
Also, let's assume the initial concentrations are:

- [A] = 10 mM
- [B] = 0 mM
- [C] = 0 mM

and the parameter values are $k_1 = 0.1 s^{-1}, k_2 = 0.05 s^{-1}$.

We can implement the model as follows:

In [ ]:
# define the 3 derivatives as a single function

def ode(t, X):
    
    # unpack the values from vector X
    a, b, c = X 
    
    # define the parameter values
    k1, k2 = 0.1, 0.05 
    
    # calculate the reaction rates
    r1 = k1 * a
    r2 = k2 * b
    
    # calculate the derivatives
    da = -r1
    db = r1 - r2
    dc = r2
    
    # return the derivatives as a single vector
    dX = [da, db, dc]
    return dX

# define the initial concentrations

X0 = [10, 0, 0]

We can now use [**solve_ivp**](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) (solve *initial value problem*) from [**SciPy**](https://scipy.org/) (a library of scientific algorithms in python) to run a simulation. 

It will integrate the **ODE** over a given **time span** starting with the **initial concentrations** at time zero:

In [ ]:
from scipy.integrate import solve_ivp

time_span = [0, 100] 

solution = solve_ivp(ode, time_span, X0)

Let's plot the simulation to see what it looks like:

In [ ]:
import matplotlib.pyplot as plt

plt.plot(solution.t, solution.y[0],
         solution.t, solution.y[1],
         solution.t, solution.y[2])
plt.legend(['A', 'B', 'C'])

🤔 You will notice that the curves are not that smooth. We can try a different integration method:

> Check the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) to learn more about integration methods.

In [ ]:
solution = solve_ivp(ode, time_span, X0, method='BDF')

plt.plot(solution.t, solution.y[0],
         solution.t, solution.y[1],
         solution.t, solution.y[2])
plt.legend(['A', 'B', 'C'])

Ok, that looks better... 😎

-----

But now we would like to see how changing the parameters would affect the solutions...

Unfortunetely, we have [*hard-coded*](https://en.wikipedia.org/wiki/Hard_coding) the parameter values inside the ODE function, which means they cannot be changed when the function is executed. 

We can instead make them arguments of the equation:

In [ ]:
def ode(t, X, k1, k2):
    
    # unpack the values from vector X
    a, b, c = X 
    
    # calculate the reaction rates
    r1 = k1 * a
    r2 = k2 * b
    
    # calculate the derivatives
    da = -r1
    db = r1 - r2
    dc = r2
    
    # return the derivatives as a single vector
    dX = [da, db, dc]
    return dX

Now we can pass the parameter values as *"additional arguments"* during simulation.

Let's run a simulation with different values:

In [ ]:
k1, k2 = 0.02, 0.01

solution = solve_ivp(ode, time_span, X0, method='BDF', args=(k1, k2))

plt.plot(solution.t, solution.y[0],
         solution.t, solution.y[1],
         solution.t, solution.y[2])
plt.legend(['A', 'B', 'C'])

Perfect, now we are warmed-up! 💪

---------

## Exercise 1:

We will now model a more complex example. Consider the following toy model of a cell:

![toy model](files/toy_model.png)

The kinetic equations are as follows:

- $r_1 = k_1$
- $r_2 = V_2\frac{A}{K_2 + A}\frac{K_I}{K_I + C}$
- $r_3 = V_3\frac{A}{K_3 + A}$
- $r_4 = k_f B - k_r D$
- $r_5 = V_5\frac{B}{K_5 + B}$
- $r_6 = k_6 C$
- $r_7 = k_7 D$

And the parameter values are:

| parameter | value | units || parameter | value | units |
| --- | --- | --- | --- | --- | --- | --- |
| $k_1$ | 5 | mM/s || $k_f$ | 0.01 | 1/s |
| $V_2$ | 30 | mM/s || $k_r$ | 0.02 | 1/s |
| $K_2$ | 1 | mM || $V_5$ | 10 | mM/s |
| $K_I$ | 1 | mM || $K_5$ | 5 | mM |
| $V_3$ | 10 | mM/s || $k_6$ | 0.5 | 1/s |
| $K_3$ | 5 | mM || $k_7$ | 0.5 | 1/s |

### 1.1 - Model building

Implement the model above as an ODE function. To speed thing up it is already partially written, just complete the code below:

> Note: We will keep only one parameter ($K_I$) as argument, the rest will be constant.

In [ ]:
def model(t, X, KI=1):

    # unpack values
    A, B, C, D = X

    # define parameters
    k1 = 5
    V2 = 30
    K2 = 1
    V3 = 10
    K3 = 5
#    kf = 
#    kr = 
#    V5 = 
#    K5 = 
#    k6 = 
#    k7 = 

    # kinetic rates
    r1 = k1
    r2 = V2 * (A / (K2 + A)) * (KI / (KI + C))
    r3 = V3 * (A / (K3 + A))
#    r4 = 
#    r5 = 
#    r6 = 
#    r7 = 
    
    # mass balance equations
    dA = r1 - r2 - r3
    dB = r2 - r4 - r5
#    dC = 
#    dD = 
    
    return [dA, dB, dC, dD]

Click below to see solution:

In [ ]:

def model(t, X, KI=1):

    A, B, C, D = X

    k1 = 5
    V2 = 30
    K2 = 1
    V3 = 10
    K3 = 5
    kf = 0.01
    kr = 0.02
    V5 = 10
    K5 = 5
    k6 = 0.5
    k7 = 0.5

    r1 = k1
    r2 = V2 * (A / (K2 + A)) * (KI / (KI + C))
    r3 = V3 * (A / (K3 + A))
    r4 = kf * B - kr * D
    r5 = V5 * (B / (K5 + B))
    r6 = k6 * C
    r7 = k7 * D
    
    dA = r1 - r2 - r3
    dB = r2 - r4 - r5
    dC = r5 - r6
    dD = r3 + r4 - r7
    
    return [dA, dB, dC, dD]

### 1.2 - Simulation

Now run a simulation using the model you just created using:
    
- initial concentrations of all compounds of 1 mM.
- a time span of 10 seconds.

Finally, plot the results.

In [ ]:
# Type your code here...

Click below to see solution:

In [ ]:

X0 = [1, 1, 1, 1]
time_span = [0, 10]

solution = solve_ivp(model, time_span, X0, method='BDF')

plt.plot(solution.t, solution.y[0],
         solution.t, solution.y[1],
         solution.t, solution.y[2],
         solution.t, solution.y[3])

plt.legend(['A', 'B', 'C', 'D'])

### 1.3 - Sensitivity analysis

If everything went well, you will see that **C** accumulates faster than the other compounds... 🤔

Going back to the diagram you can see that there is a regulatory feedback loop from **C** to **R2** to prevent accumulation. This is controlled by the term ($K_I/(K_I + C)$) in rate *r2*.

The inhibition constant $K_I$ (mM) represents the affinity of the inhibitor to the enzyme. The lower the value, the higher the affinity, and the stronger the inhibition will be.

Let's test the final concentration of **C** (at 10 sec) with different affinity constants:

- Run a series of simulations for different values of $K_I$ (0.001, 0.01, 0.1, 1, 10, 100, 1000)
- Plot the final concentration of **C** as a function of $K_I$

> Tip (1): the solution object from solve_ivp contains a matrix (*y*) with format "variables x timepoints".
> You can use `solution.y[2,-1]` to get the final value of *C*.

> Tip (2): you can use `plt.semilogx` instead of `plt.plot` to have a log-scale on the x-axis

In [ ]:
# type your code here...

Click below to see solution:

In [ ]:

KIs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
C_final = []

for ki in KIs:
    solution = solve_ivp(model, [0, 10], X0, args=[ki], method='BDF')
    C_final.append(solution.y[2,-1])
    
plt.semilogx(KIs, C_final, 'o-')
plt.xlabel('KI')
plt.ylabel('[C]')

🧠 Reflect on the result. Is this what you expected?

---------

## Exercise 2 - Parameter estimation

Now let's learn how we can use experimental data to calibrate a model (using *parameter estimation*).

We will use some (fake) metabolomics data:

In [ ]:
import pandas as pd

df = pd.read_csv('files/metabolomics.csv')
df

Let's compare this experimental data with the model simulation:

In [ ]:
X0 = [1, 1, 1, 1]
time_span = [0, 10]

solution = solve_ivp(model, time_span, X0, method='BDF')

plt.plot(solution.t, solution.y[0])
plt.plot(solution.t, solution.y[1])
plt.plot(solution.t, solution.y[2])
plt.plot(solution.t, solution.y[3])

plt.gca().set_prop_cycle(None) # reset color cycle

plt.plot(df['time'], df['A'], 'o--')
plt.plot(df['time'], df['B'], 'o--')
plt.plot(df['time'], df['C'], 'o--')
plt.plot(df['time'], df['D'], 'o--')

plt.legend(['A', 'B', 'C', 'D'])

As you could see, there is quite a difference between the experimental data and the simulation result.

**Note:** We can choose to calibrate *all* or just *some* of the kinetic parameters in the model. In this example, reactions R2, R3 and R5 use Michaelis-Menten kinetics, a type of equation that contains two parameters $V_{max}$, the maximum rate achieved when the enzyme is saturated, and $K_M$, that represents the substrate binding affinity. Remember that $V_{max}$ depends on the enzyme abundance, it can also be represented as $k_{cat}[E]$, where $k_{cat}$ is the catalytic rate constant. Therefore, while $k_{cat}$ and $K_M$ are enzyme-specific constants, the enzyme concentration $[E]$ can be controled through gene regulation, making $V_{max}$ a condition-dependent parameter.

Let's start by changing the implementation of our model, so that we can more easily change the $V_{max}$ values (V2, V3, and V5):

In [ ]:
def model(t, X, V2=30, V3=10, V5=10):

    A, B, C, D = X

    k1 = 5
    K2 = 1
    KI = 1
    K3 = 5
    kf = 0.01
    kr = 0.02
    K5 = 5
    k6 = 0.5
    k7 = 0.5

    r1 = k1
    r2 = V2 * (A / (K2 + A)) * (KI / (KI + C))
    r3 = V3 * (A / (K3 + A))
    r4 = kf * B - kr * D
    r5 = V5 * (B / (K5 + B))
    r6 = k6 * C
    r7 = k7 * D
    
    dA = r1 - r2 - r3
    dB = r2 - r4 - r5
    dC = r5 - r6
    dD = r3 + r4 - r7
    
    return [dA, dB, dC, dD]

To estimate the parameters we need to minimize the total difference between the experimental and simulated data points. 

Let's create a function that, given the parameter estimates as input, calculates that difference.

In [ ]:
def error(params):
    
    # solve ODE and evaluate at same time intervals as data
    solution = solve_ivp(model, time_span, X0, args=params, method='BDF', t_eval=df['time'])
    
    diff_a = sum(abs(solution.y[0] - df['A']))
    diff_b = sum(abs(solution.y[1] - df['B']))
    diff_c = sum(abs(solution.y[2] - df['C']))
    diff_d = sum(abs(solution.y[3] - df['D']))
    
    return diff_a + diff_b + diff_c + diff_d

### 2.1

Use function [**minimize**](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) from *scipy* to find parameter values that minimize our *error* function. 

> **Tip 1**: When you read the documentation, please note the `x0` in this case corresponds to the initial parameter estimates, and `result.x` correspond to the re-estimated parameters.

> **Tip 2**: *minimize* is actually a common wrapper for different minimization methods. Some of these might fail to converge to the correct solution. One way to help them converge is to provide some limits (bounds) to the expected parameter values. In this case, assume that each parameter is in the interval (0, 100) mM/s.

In [ ]:
from scipy.optimize import minimize

# type your code here...

Click below to see solution:

In [ ]:

params0 = [30, 10, 10]

bounds = [
    (0, 100),
    (0, 100),
    (0, 100)
]

result = minimize(error, params0, bounds=bounds)

V2, V3, V5 = result.x #unpacking values from solution

print(f'V2 = {V2:.4g}')
print(f'V3 = {V3:.4g}')
print(f'V5 = {V5:.4g}')

### 2.2 

Now simulate the model again using the parameters you just estimated and compare with the experimental data.

> **Tip 1:** you can just copy-paste the "simulate and compare" code from a few cells above

> **Tip 2:** you can change the simulation parameters using the `args` argument in *solve_ivp*

In [ ]:
# type your code here

Click below to see the solution:

In [ ]:

X0 = [1, 1, 1, 1]
time_span = [0, 10]

solution = solve_ivp(model, time_span, X0, args=result.x, method='BDF')

plt.plot(solution.t, solution.y[0])
plt.plot(solution.t, solution.y[1])
plt.plot(solution.t, solution.y[2])
plt.plot(solution.t, solution.y[3])

plt.gca().set_prop_cycle(None) # reset color cycle

plt.plot(df['time'], df['A'], 'o--')
plt.plot(df['time'], df['B'], 'o--')
plt.plot(df['time'], df['C'], 'o--')
plt.plot(df['time'], df['D'], 'o--')

plt.legend(['A', 'B', 'C', 'D'])

----------

## Wrap-up

This was a VERY extensive tutorial. 

You learned to create a mechanistic model of a biological system using ODEs, to run simulations with your model, test the sensitivity of the results to the parameters, and to re-calibrate the model to match experimental observations. 🤯

It is ok if you had to look at the solutions. The most important is to understand how a diagram of a biological system can be transformed into a set of equations that can be used to predict (and understand how to manipulate) the behavior of that system. 

Great job! 😉